In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import random_uuid
import pprint
import argparse

from utils import load_question

from graph import DataExtractor

In [2]:
# .env 파일 로드
load_dotenv(dotenv_path=".env")

# API 키 가져오기
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")

# LangSmith 추적 기능을 활성화합니다. (선택적)
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Multi-agent Collaboration"

In [3]:
# 4개의 질문 입력
questions = load_question()


In [5]:
total_outputs = {}
for file_num in [39]:
    print(f"#####   {file_num}번째 논문    #####")
    # `file_num` 키 초기화
    if file_num not in total_outputs:
        total_outputs[file_num] = {}

    for j in [0.4, 0.6, 0.8]:
        voltai_graph = DataExtractor(
            # folder_path, 
            file_number=file_num,
            temperature = j
        ).graph
        print(j)
        # config 설정(재귀 최대 횟수, thread_id)
        config = RunnableConfig(
            recursion_limit=20, 
            configurable={"thread_id": random_uuid()}
        )

        # 4개의 질문에 대해 그래프 실행 및 출력
        results = []
        for i, question in enumerate(questions):
            print(f"{i+1}번째 질문")
            result = voltai_graph.invoke(
                input={"question": question},
                config=config,
            )
            results.append(result)

        # `i` 값에 따른 결과 저장
        total_outputs[file_num][f"{j}"] = results

#####   39번째 논문    #####
    vectordb-paper_039을 ./vectordb/vectordb-paper_039에 저장했습니다.
0.4
1번째 질문
        RELEVANCE CHECK : yes
    vectordb-paper_039을 불러왔습니다.
0.6
1번째 질문
        RELEVANCE CHECK : yes
    vectordb-paper_039을 불러왔습니다.
0.8
1번째 질문
        RELEVANCE CHECK : yes


In [25]:
total_outputs[39]["0.4"][0]['answer']

[{'CAM (Cathode Active Material)': {'Stoichiometry information': {'NCM-622': {'Li ratio': 'None',
     'Ni ratio': 'None',
     'Co ratio': 'None',
     'Mn ratio': 'None',
     'O ratio': 'None'},
    'ZrO2-NCM-622 (Z622)': {'Li ratio': 'None',
     'Ni ratio': 'None',
     'Co ratio': 'None',
     'Mn ratio': 'None',
     'O ratio': 'None'}},
   'Whether or not commercial NCM was used for each sample (Stoichiometry information in order)': ['None',
    'None'],
   'Lithium source': 'LiOH·H2O',
   'Synthesis method': 'solid-phase method',
   'Describe the crystallization method, such as Hydrothermal, Sintering, or any other technique used during the process.': 'spray drying',
   'What is the Crystallization final temperature in degree of Celcius used in the process? (e.g., calcination or sintering) mentioned for the crystallization stage.': '780',
   'What is the time duration for the final crystallization process, including any calcination or sintering stages? Specify the hours.': '10

In [21]:
total_outputs[39]["0.6"][0]['answer'][0]['CAM (Cathode Active Material)'].values()

dict_values([{'NCM-622': {'Li ratio': 'None', 'Ni ratio': 'None', 'Co ratio': 'None', 'Mn ratio': 'None', 'O ratio': 'None'}, 'ZrO2-NCM-622 (Z622)': {'Li ratio': 'None', 'Ni ratio': 'None', 'Co ratio': 'None', 'Mn ratio': 'None', 'O ratio': 'None'}}, ['None', 'None'], 'LiOH·H2O', 'solid-phase method', 'Spray drying and calcination', '780', '10', 'W6+', 'None', 'None'])

In [23]:
a =str(total_outputs[39]["0.8"][0]['answer'][0])

In [24]:
a

"{'CAM (Cathode Active Material)': {'Stoichiometry information': {'NCM-622': {'Li ratio': 'None', 'Ni ratio': 'None', 'Co ratio': 'None', 'Mn ratio': 'None', 'O ratio': 'None'}, 'ZrO2-NCM-622 (Z622)': {'Li ratio': 'None', 'Ni ratio': 'None', 'Co ratio': 'None', 'Mn ratio': 'None', 'O ratio': 'None'}}, 'Whether or not commercial NCM was used for each sample (Stoichiometry information in order)': ['None', 'None'], 'Lithium source': 'LiOH·H2O', 'Synthesis method': 'solid-phase method', 'Describe the crystallization method, such as Hydrothermal, Sintering, or any other technique used during the process.': 'calcination', 'What is the Crystallization final temperature in degree of Celcius used in the process? (e.g., calcination or sintering) mentioned for the crystallization stage.': '780', 'What is the time duration for the final crystallization process, including any calcination or sintering stages? Specify the hours.': '10', 'Doping': 'W6+', 'Coating': 'None', 'Additional treatment': 'Non